In [2]:
import socket
import sys
import requests
import requests_oauthlib
import json

In [4]:
ACCESS_TOKEN = "529681232-aRazWzdsGEtrm7ptqxjNBKhsn7zP9SyUqyBToSBA"
ACCESS_TOKEN_SECRET = "x8JxKR1qQIURmpZPSruuUNuzcC3vPd5REvZaIVkLzUOi0"
CONSUMER_KEY = "nkxT4EBq7zHwmvQkomlxLcc1D"
CONSUMER_SECRET = "TDgE2sdCcf1eTp0QUWvz46Ntel1DE6ESivJQGHiMBy3iwGcB3s"

my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
BASE_URL = "https://stream.twitter.com"

In [7]:
def get_tweets():
    url = BASE_URL+"/1.1/statuses/filter.json"
    query_params = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_params])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [9]:
def send_tweets_to_socket(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            print('Tweet text:'+ tweet_text)
            tcp_connection.send(tweet_text+'\n')
        except:
            e = sys.exc_info()[0]
            print("Error %s" %e)

In [ ]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected to TCP. Receiving tweets...")
response = get_tweets()
send_tweets_to_socket(response, conn)

Waiting for TCP connection...
